In [1]:
# # import the needed libraries
# import tensorflow as tf
# tf.keras.backend.set_learning_phase(0) #use this if we have batch norm layer in our network
# from tensorflow.python.keras.models import load_model

from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

In [2]:
from keras.models import load_model
model = load_model('./models/train_mrl_50.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

[<tf.Tensor 'activation_2/Sigmoid:0' shape=(?, 1) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(?, 32, 32, 1) dtype=float32>]


In [3]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 45 variables.
Converted 45 variables to const ops.


In [4]:
tf.train.write_graph(frozen_graph, "models", "TF_another5_mrl.pb", as_text=False)

'models/TF_another5_mrl.pb'

In [5]:
# # path we wanna save our converted TF-model
# #MODEL_PATH = "./model/tensorflow/big/model1"
# MODEL_PATH = "./models"

# # load the Keras model
# #model = load_model('./model/modelLeNet5.h5')
# model = load_model('./models/train_mrl_50.h5')

# # save the model to Tensorflow model
# saver = tf.train.Saver()
# sess = tf.keras.backend.get_session()
# save_path = saver.save(sess, MODEL_PATH)

# print("Keras model is successfully converted to TF model in "+MODEL_PATH)